In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pprint
import os
from sklearn.decomposition import PCA
from sklearn.mixture import GaussianMixture
from sklearn import metrics

pd.set_option('display.max_colwidth',100000) #https://stackoverflow.com/questions/54692405/output-truncation-in-google-colab
pd.set_option('display.max_rows', 100000)

# Below imports are used to print out pretty pandas dataframes
from IPython.display import display, HTML

In [ ]:
from datetime import datetime
def convert_time(x):
  date_time_str = x
  if date_time_str[-1] == ':':
    date_time_str += '00'
  date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S')
  return date_time_obj

In [ ]:
basic_table = pd.read_pickle('Downloads/basic_table.pkl')
physio = pd.read_csv('Downloads/physiology.csv')
final_adult_patients = pd.read_csv('Downloads/final_adult_patients.csv')
vasoactive = pd.read_csv('Downloads/vasoactive.csv')
rrt = pd.read_csv('Downloads/rrt.csv')

In [ ]:
print(type(basic_table['pO2_time'].iloc[0]))

In [ ]:
physio.charttime = physio.charttime.apply(convert_time)
basic_table.pO2_time = basic_table.pO2_time.apply(convert_time)
physio.head(5)

In [ ]:
print(type(basic_table['pO2_time'].iloc[0]))

In [ ]:
basic_table.columns.values

In [ ]:
time = pd.DataFrame({'stay_id': basic_table['stay_id'], 'pO2_time': basic_table['pO2_time'], 'dicom':basic_table['dicom'] })

In [ ]:
phys = physio.merge(time, how='inner', on='stay_id')
phys = phys.drop(['Unnamed: 0'], axis=1)
phys.head(50)

In [ ]:
print(phys['dicom'].nunique())

In [ ]:
phys['time_diff'] = phys.pO2_time - phys.charttime
phys = phys[phys['time_diff'] > pd.Timedelta(0)]

phys['time_diff'] = phys['time_diff'] / pd.Timedelta(minutes=1)
phys['min_time_diff'] = phys.groupby('dicom')['time_diff'].transform('min')

phys = phys.where(phys['time_diff'] == phys['min_time_diff'])

phys = phys[phys['dicom'].notna()]


In [ ]:
print(len(phys))
print(phys['dicom'].nunique())
print(phys['subject_id'].nunique())


In [ ]:
phys = phys.reset_index()
phys = phys.drop(['index'], axis=1)
phys.head(5)

In [ ]:
fio2 = pd.DataFrame({'fio2': final_adult_patients['converted_fiO2'], 'dicom':final_adult_patients['dicom'] })

In [ ]:
physio = basic_table.merge(fio2, how='inner', on='dicom')
physio.head(100)

In [ ]:
physio = physio.drop_duplicates(subset='dicom', ignore_index=True)

In [ ]:
physio.head(100
           )

In [ ]:
print(len(physio))
print(physio['dicom'].nunique())
print(physio['stay_id'].nunique())


In [ ]:
phys = phys.drop(['time_diff','min_time_diff',], axis=1)

In [ ]:
ph = physio.merge(phys, how='left', on='dicom')
ph.head(50)

In [ ]:
print(len(ph))
print(ph['dicom'].nunique())
print(ph['stay_id_x'].nunique())

In [ ]:
ph.columns.values


In [ ]:
ph =ph.drop(['fio2_y', 'pO2_time_y', 'stay_id_y','charttime',], axis=1)
ph = ph.sort_values(by=['label'])
ph.head(20)

In [ ]:
ph['driving_pressure']= ph.plateau_pressure - ph.peep
ph = ph.reset_index()
ph.head(5)

In [ ]:
ph =ph.drop(['subject_id'], axis=1)

In [ ]:
ph =ph.drop(['index'], axis=1)

In [ ]:
print(ph['driving_pressure'].nunique())

In [ ]:
# ph.to_excel('Downloads/phyiology.xlsx')

In [ ]:
vasoactive = vasoactive.drop(['Unnamed: 0'], axis=1)
vasoactive.columns.values

In [ ]:
vasoactive = vasoactive.groupby('stay_id').sum()

vasoactive.reset_index(inplace=True)

vasoactive.head(10)

In [ ]:
time.columns.values

In [ ]:
vas = time.merge(vasoactive, how='left', on='stay_id')
print(len(vas))
vas.head(10)

In [ ]:
organ_support = basic_table.merge(vas, how='left', on='dicom')
print(len(organ_support))

In [ ]:
organ_support['vasopressors'] = organ_support['dopamine'] + organ_support['epinephrine'] + organ_support['norepinephrine'] + organ_support['phenylephrine'] + organ_support['vasopressin']

organ_support['inotropes'] = organ_support['dopamine'] + organ_support['epinephrine'] + organ_support['norepinephrine'] + organ_support['milrinone'] + organ_support['dobutamine']
organ_support.head(5)

In [ ]:
# organ_support.to_excel('Downloads/organ_support.xlsx')

In [ ]:
rrt = rrt.drop(['Unnamed: 0','charttime', 'dialysis_type', 'dialysis_active' ], axis = 1)
rrt = rrt.groupby('stay_id').sum()
vasoactive.reset_index(inplace=True)
rrt.columns.values

In [ ]:
rrt.head(20)

In [ ]:
print(len (rrt))
print(rrt['stay_id'].nunique())

In [ ]:
rrtt = basic_table.merge(rrt, how='left', on='stay_id')
print(len (rrtt))
print(rrtt['stay_id'].nunique())

In [ ]:
rrtt.head(50)

In [ ]:
# rrtt.to_excel('Downloads/rrt2.xlsx')